# Chapter 32: Accessibility (a11y)

Building accessible web applications ensures that users with disabilities—including those using screen readers, keyboard navigation, or assistive technologies—can fully interact with your Next.js application. Accessibility is not merely compliance with legal standards but a fundamental aspect of inclusive design that improves usability for everyone. The App Router's server components and streaming architecture require specific patterns to maintain accessibility while delivering dynamic content.

By the end of this chapter, you'll master implementing semantic HTML structures, using ARIA attributes appropriately for dynamic content, managing keyboard navigation and focus traps, creating screen reader announcements for live regions, ensuring sufficient color contrast and text sizing, building accessible form components with error handling, and integrating automated accessibility testing into your workflow.

## 32.1 Semantic HTML Foundations

Use proper HTML elements to convey meaning and structure without relying solely on ARIA attributes.

### Document Structure and Landmarks

```typescript
// app/layout.tsx
import { Inter } from 'next/font/google';
import { type ReactNode } from 'react';

const inter = Inter({ subsets: ['latin'] });

export default function RootLayout({ 
  children,
  params: { locale }
}: { 
  children: ReactNode;
  params: { locale: string };
}) {
  return (
    <html lang={locale}>
      <body className={inter.className}>
        {/* Skip to main content link for keyboard users */}
        <a 
          href="#main-content"
          className="sr-only focus:not-sr-only focus:absolute focus:top-4 focus:left-4 focus:z-50 focus:px-4 focus:py-2 focus:bg-blue-600 focus:text-white focus:rounded"
        >
          Skip to main content
        </a>
        
        <header role="banner" className="border-b">
          <nav role="navigation" aria-label="Main navigation">
            {/* Navigation content */}
          </nav>
        </header>
        
        <main id="main-content" role="main" className="min-h-screen">
          {children}
        </main>
        
        <footer role="contentinfo" className="border-t">
          {/* Footer content */}
        </footer>
      </body>
    </html>
  );
}
```

### Accessible Navigation Components

```typescript
// components/navigation.tsx
'use client';

import Link from 'next/link';
import { usePathname } from 'next/navigation';
import { useState } from 'react';

export function Navigation() {
  const pathname = usePathname();
  const [mobileMenuOpen, setMobileMenuOpen] = useState(false);
  
  const navItems = [
    { href: '/', label: 'Home' },
    { href: '/courses', label: 'Courses' },
    { href: '/about', label: 'About' },
  ];

  return (
    <nav aria-label="Main">
      {/* Mobile menu button */}
      <button
        type="button"
        aria-expanded={mobileMenuOpen}
        aria-controls="mobile-menu"
        aria-label={mobileMenuOpen ? "Close menu" : "Open menu"}
        onClick={() => setMobileMenuOpen(!mobileMenuOpen)}
        className="md:hidden p-2"
      >
        {mobileMenuOpen ? <CloseIcon /> : <MenuIcon />}
      </button>

      {/* Desktop navigation */}
      <ul className="hidden md:flex gap-6">
        {navItems.map((item) => {
          const isActive = pathname === item.href;
          
          return (
            <li key={item.href}>
              <Link
                href={item.href}
                aria-current={isActive ? 'page' : undefined}
                className={`px-3 py-2 rounded ${
                  isActive 
                    ? 'bg-blue-100 text-blue-800 font-medium' 
                    : 'text-gray-600 hover:text-gray-900'
                }`}
              >
                {item.label}
              </Link>
            </li>
          );
        })}
      </ul>

      {/* Mobile navigation */}
      {mobileMenuOpen && (
        <div 
          id="mobile-menu"
          role="menu"
          className="md:hidden absolute top-16 left-0 right-0 bg-white border-b shadow-lg"
        >
          <ul className="py-2">
            {navItems.map((item) => (
              <li key={item.href} role="none">
                <Link
                  href={item.href}
                  role="menuitem"
                  className="block px-4 py-3 hover:bg-gray-50"
                  onClick={() => setMobileMenuOpen(false)}
                >
                  {item.label}
                </Link>
              </li>
            ))}
          </ul>
        </div>
      )}
    </nav>
  );
}
```

## 32.2 ARIA Attributes and Patterns

Implement ARIA attributes when HTML semantics alone are insufficient, particularly for dynamic content and custom components.

### Accessible Modal Dialogs

```typescript
// components/modal.tsx
'use client';

import { useEffect, useRef } from 'react';
import { createPortal } from 'react-dom';
import { useId } from 'react';
import { FocusTrap } from './focus-trap';

interface ModalProps {
  isOpen: boolean;
  onClose: () => void;
  title: string;
  description?: string;
  children: React.ReactNode;
}

export function Modal({ isOpen, onClose, title, description, children }: ModalProps) {
  const titleId = useId();
  const descriptionId = useId();
  const previousActiveElement = useRef<HTMLElement | null>(null);
  
  useEffect(() => {
    if (isOpen) {
      // Store the element that triggered the modal
      previousActiveElement.current = document.activeElement as HTMLElement;
      
      // Prevent body scroll
      document.body.style.overflow = 'hidden';
      
      // Handle escape key
      const handleEscape = (e: KeyboardEvent) => {
        if (e.key === 'Escape') onClose();
      };
      
      document.addEventListener('keydown', handleEscape);
      return () => {
        document.removeEventListener('keydown', handleEscape);
        document.body.style.overflow = '';
        // Restore focus when closing
        previousActiveElement.current?.focus();
      };
    }
  }, [isOpen, onClose]);

  if (!isOpen) return null;

  return createPortal(
    <div
      role="dialog"
      aria-modal="true"
      aria-labelledby={titleId}
      aria-describedby={description ? descriptionId : undefined}
      className="fixed inset-0 z-50 flex items-center justify-center"
    >
      {/* Backdrop */}
      <div 
        className="absolute inset-0 bg-black/50"
        onClick={onClose}
        aria-hidden="true"
      />
      
      {/* Modal content */}
      <FocusTrap>
        <div className="relative bg-white rounded-lg shadow-xl max-w-lg w-full mx-4 p-6">
          <div className="flex items-center justify-between mb-4">
            <h2 id={titleId} className="text-xl font-bold">
              {title}
            </h2>
            <button
              type="button"
              onClick={onClose}
              aria-label="Close dialog"
              className="p-2 hover:bg-gray-100 rounded-full"
            >
              <CloseIcon className="w-5 h-5" aria-hidden="true" />
            </button>
          </div>
          
          {description && (
            <p id={descriptionId} className="text-gray-600 mb-4">
              {description}
            </p>
          )}
          
          {children}
        </div>
      </FocusTrap>
    </div>,
    document.body
  );
}

// components/focus-trap.tsx
'use client';

import { useEffect, useRef } from 'react';

interface FocusTrapProps {
  children: React.ReactNode;
}

export function FocusTrap({ children }: FocusTrapProps) {
  const containerRef = useRef<HTMLDivElement>(null);
  
  useEffect(() => {
    const container = containerRef.current;
    if (!container) return;
    
    const focusableElements = container.querySelectorAll<HTMLElement>(
      'button, [href], input, select, textarea, [tabindex]:not([tabindex="-1"])'
    );
    
    const firstElement = focusableElements[0];
    const lastElement = focusableElements[focusableElements.length - 1];
    
    // Focus first element on mount
    firstElement?.focus();
    
    const handleTabKey = (e: KeyboardEvent) => {
      if (e.key !== 'Tab') return;
      
      if (e.shiftKey) {
        if (document.activeElement === firstElement) {
          lastElement?.focus();
          e.preventDefault();
        }
      } else {
        if (document.activeElement === lastElement) {
          firstElement?.focus();
          e.preventDefault();
        }
      }
    };
    
    container.addEventListener('keydown', handleTabKey);
    return () => container.removeEventListener('keydown', handleTabKey);
  }, []);
  
  return <div ref={containerRef}>{children}</div>;
}
```

### Accessible Tabs Component

```typescript
// components/tabs.tsx
'use client';

import { useState, useId } from 'react';

interface Tab {
  id: string;
  label: string;
  content: React.ReactNode;
}

interface TabsProps {
  tabs: Tab[];
}

export function AccessibleTabs({ tabs }: TabsProps) {
  const [activeTab, setActiveTab] = useState(0);
  const tabListId = useId();
  
  const handleKeyDown = (e: React.KeyboardEvent, index: number) => {
    let nextIndex = activeTab;
    
    switch (e.key) {
      case 'ArrowRight':
        nextIndex = (activeTab + 1) % tabs.length;
        break;
      case 'ArrowLeft':
        nextIndex = (activeTab - 1 + tabs.length) % tabs.length;
        break;
      case 'Home':
        nextIndex = 0;
        break;
      case 'End':
        nextIndex = tabs.length - 1;
        break;
      default:
        return;
    }
    
    e.preventDefault();
    setActiveTab(nextIndex);
    // Focus the new tab button
    document.getElementById(`tab-${tabs[nextIndex].id}`)?.focus();
  };

  return (
    <div>
      <div 
        role="tablist" 
        aria-label="Course sections"
        className="flex border-b gap-1"
      >
        {tabs.map((tab, index) => (
          <button
            key={tab.id}
            id={`tab-${tab.id}`}
            role="tab"
            aria-selected={activeTab === index}
            aria-controls={`panel-${tab.id}`}
            tabIndex={activeTab === index ? 0 : -1}
            onClick={() => setActiveTab(index)}
            onKeyDown={(e) => handleKeyDown(e, index)}
            className={`px-4 py-2 border-b-2 font-medium ${
              activeTab === index 
                ? 'border-blue-600 text-blue-600' 
                : 'border-transparent text-gray-600 hover:text-gray-900'
            }`}
          >
            {tab.label}
          </button>
        ))}
      </div>
      
      {tabs.map((tab, index) => (
        <div
          key={tab.id}
          id={`panel-${tab.id}`}
          role="tabpanel"
          aria-labelledby={`tab-${tab.id}`}
          hidden={activeTab !== index}
          className="p-4"
          tabIndex={0}
        >
          {tab.content}
        </div>
      ))}
    </div>
  );
}
```

## 32.3 Screen Reader Support and Live Regions

Ensure dynamic content changes are announced to assistive technologies.

### Announcer Component

```typescript
// components/announcer.tsx
'use client';

import { useEffect, useState } from 'react';

interface AnnouncerProps {
  message: string;
  priority?: 'polite' | 'assertive';
}

export function Announcer({ message, priority = 'polite' }: AnnouncerProps) {
  const [announcement, setAnnouncement] = useState('');

  useEffect(() => {
    // Clear first, then set new message (ensures screen readers detect change)
    setAnnouncement('');
    const timer = setTimeout(() => setAnnouncement(message), 100);
    return () => clearTimeout(timer);
  }, [message]);

  return (
    <div
      aria-live={priority}
      aria-atomic="true"
      className="sr-only"
      role="status"
      aria-relevant="additions text"
    >
      {announcement}
    </div>
  );
}

// Usage in forms
// components/form-with-announcements.tsx
'use client';

import { useState } from 'react';
import { Announcer } from './announcer';

export function ContactForm() {
  const [status, setStatus] = useState<'idle' | 'submitting' | 'success' | 'error'>('idle');
  const [announcement, setAnnouncement] = useState('');

  const handleSubmit = async (e: React.FormEvent) => {
    e.preventDefault();
    setStatus('submitting');
    setAnnouncement('Submitting form, please wait');
    
    try {
      await submitForm();
      setStatus('success');
      setAnnouncement('Form submitted successfully. We will contact you soon.');
    } catch (error) {
      setStatus('error');
      setAnnouncement('Error submitting form. Please check your inputs and try again.');
    }
  };

  return (
    <>
      <Announcer message={announcement} priority="assertive" />
      
      <form onSubmit={handleSubmit} aria-busy={status === 'submitting'}>
        {/* Form fields */}
        <button 
          type="submit" 
          disabled={status === 'submitting'}
          aria-describedby="form-status"
        >
          {status === 'submitting' ? 'Submitting...' : 'Submit'}
        </button>
        
        {status === 'error' && (
          <div id="form-status" role="alert" className="text-red-600 mt-2">
            There was an error submitting the form. Please try again.
          </div>
        )}
      </form>
    </>
  );
}
```

### Accessible Data Tables

```typescript
// components/data-table.tsx
interface Column<T> {
  key: string;
  header: string;
  render: (item: T) => React.ReactNode;
}

interface DataTableProps<T> {
  columns: Column<T>[];
  data: T[];
  caption?: string;
}

export function AccessibleDataTable<T extends { id: string }>({ 
  columns, 
  data,
  caption 
}: DataTableProps<T>) {
  return (
    <div className="overflow-x-auto">
      <table className="w-full border-collapse">
        {caption && <caption className="text-left py-2 font-medium">{caption}</caption>}
        <thead>
          <tr className="border-b">
            {columns.map((col) => (
              <th 
                key={col.key} 
                scope="col"
                className="text-left py-3 px-4 font-semibold"
              >
                {col.header}
              </th>
            ))}
          </tr>
        </thead>
        <tbody>
          {data.map((item, index) => (
            <tr key={item.id} className="border-b hover:bg-gray-50">
              {columns.map((col, colIndex) => (
                <td key={col.key} className="py-3 px-4">
                  {colIndex === 0 ? (
                    <th scope="row" className="font-normal">
                      {col.render(item)}
                    </th>
                  ) : (
                    col.render(item)
                  )}
                </td>
              ))}
            </tr>
          ))}
        </tbody>
      </table>
    </div>
  );
}
```

## 32.4 Accessible Forms

Implement forms with proper labeling, error handling, and validation announcements.

### Accessible Form Components

```typescript
// components/form-field.tsx
'use client';

import { useId } from 'react';

interface FormFieldProps {
  label: string;
  error?: string;
  hint?: string;
  required?: boolean;
  children: (props: { 
    id: string; 
    'aria-describedby'?: string;
    'aria-invalid'?: boolean;
    'aria-required'?: boolean;
  }) => React.ReactNode;
}

export function FormField({ label, error, hint, required, children }: FormFieldProps) {
  const id = useId();
  const errorId = `${id}-error`;
  const hintId = `${id}-hint`;
  
  const describedBy = [
    hint ? hintId : null,
    error ? errorId : null
  ].filter(Boolean).join(' ') || undefined;

  return (
    <div className="mb-4">
      <label 
        htmlFor={id}
        className="block text-sm font-medium mb-1"
      >
        {label}
        {required && <span aria-label="required" className="text-red-500 ml-1">*</span>}
      </label>
      
      {children({
        id,
        'aria-describedby': describedBy,
        'aria-invalid': !!error,
        'aria-required': required,
      })}
      
      {hint && !error && (
        <p id={hintId} className="text-sm text-gray-500 mt-1">
          {hint}
        </p>
      )}
      
      {error && (
        <p id={errorId} role="alert" className="text-sm text-red-600 mt-1">
          {error}
        </p>
      )}
    </div>
  );
}

// Usage
// components/login-form.tsx
'use client';

import { FormField } from './form-field';
import { useForm } from 'react-hook-form';
import { zodResolver } from '@hookform/resolvers/zod';
import { z } from 'zod';

const schema = z.object({
  email: z.string().email('Please enter a valid email address'),
  password: z.string().min(8, 'Password must be at least 8 characters'),
});

export function LoginForm() {
  const { register, handleSubmit, formState: { errors } } = useForm({
    resolver: zodResolver(schema),
  });

  return (
    <form onSubmit={handleSubmit(onSubmit)} noValidate>
      <FormField
        label="Email address"
        error={errors.email?.message}
        required
      >
        {(props) => (
          <input
            {...props}
            {...register('email')}
            type="email"
            className="w-full px-3 py-2 border rounded focus:ring-2 focus:ring-blue-500 focus:outline-none"
          />
        )}
      </FormField>
      
      <FormField
        label="Password"
        error={errors.password?.message}
        hint="Must be at least 8 characters"
        required
      >
        {(props) => (
          <input
            {...props}
            {...register('password')}
            type="password"
            className="w-full px-3 py-2 border rounded focus:ring-2 focus:ring-blue-500 focus:outline-none"
          />
        )}
      </FormField>
      
      <button 
        type="submit"
        className="w-full py-2 bg-blue-600 text-white rounded hover:bg-blue-700 focus:ring-2 focus:ring-offset-2 focus:ring-blue-500"
      >
        Sign In
      </button>
    </form>
  );
}
```

## 32.5 Automated Accessibility Testing

Integrate accessibility checks into your development workflow and CI/CD pipeline.

### Testing Setup with jest-axe

```typescript
// __tests__/components/button.test.tsx
import { render, screen } from '@testing-library/react';
import { axe, toHaveNoViolations } from 'jest-axe';
import { Button } from '@/components/button';

expect.extend(toHaveNoViolations);

describe('Button', () => {
  it('should have no accessibility violations', async () => {
    const { container } = render(<Button>Click me</Button>);
    const results = await axe(container);
    expect(results).toHaveNoViolations();
  });

  it('should be accessible when disabled', async () => {
    const { container } = render(<Button disabled>Disabled</Button>);
    const results = await axe(container);
    expect(results).toHaveNoViolations();
  });
});

// __tests__/pages/home.test.tsx
import { render } from '@testing-library/react';
import { axe } from 'jest-axe';
import Home from '@/app/page';

describe('Home Page', () => {
  it('should have no accessibility violations', async () => {
    const { container } = render(<Home />);
    const results = await axe(container, {
      rules: {
        // Disable color contrast for tests if using custom colors
        'color-contrast': { enabled: false },
      },
    });
    expect(results).toHaveNoViolations();
  });
});
```

### Storybook A11y Addon

```typescript
// .storybook/main.ts
const config = {
  addons: [
    '@storybook/addon-essentials',
    '@storybook/addon-a11y', // Accessibility testing in Storybook
  ],
};

// components/button.stories.tsx
import type { Meta, StoryObj } from '@storybook/react';
import { Button } from './button';

const meta: Meta<typeof Button> = {
  component: Button,
  parameters: {
    a11y: {
      // Override automatic checks
      config: {
        rules: [
          { id: 'button-name', enabled: true },
        ],
      },
    },
  },
};

export default meta;

export const Default: StoryObj = {
  render: () => <Button>Accessible Button</Button>,
};

export const IconOnly: StoryObj = {
  render: () => (
    <Button aria-label="Close dialog">
      <CloseIcon />
    </Button>
  ),
};
```

### CI Integration

```yaml
# .github/workflows/a11y.yml
name: Accessibility Tests

on: [push, pull_request]

jobs:
  test:
    runs-on: ubuntu-latest
    steps:
      - uses: actions/checkout@v3
      
      - name: Setup Node.js
        uses: actions/setup-node@v3
        with:
          node-version: '20'
          
      - name: Install dependencies
        run: npm ci
        
      - name: Run accessibility tests
        run: npm run test:a11y
        
      - name: Run Lighthouse CI
        run: |
          npm install -g @lhci/cli@0.12.x
          lhci autorun
        env:
          LHCI_GITHUB_APP_TOKEN: ${{ secrets.LHCI_GITHUB_APP_TOKEN }}

# lighthouserc.js
module.exports = {
  ci: {
    collect: {
      url: ['http://localhost:3000/'],
      startServerCommand: 'npm run start',
    },
    assert: {
      assertions: {
        'categories.accessibility': ['error', { minScore: 0.95 }],
        'categories.best-practices': ['warn', { minScore: 0.9 }],
      },
    },
  },
};
```

## Key Takeaways from Chapter 32

1. **Semantic HTML**: Use native HTML elements (`<button>`, `<nav>`, `<main>`, `<article>`) before reaching for ARIA attributes. Implement skip links for keyboard users to bypass navigation and reach main content quickly.

2. **ARIA Usage**: Apply ARIA attributes only when HTML semantics are insufficient. Use `aria-expanded` for dropdowns, `aria-current="page"` for active navigation, `aria-live` regions for dynamic content announcements, and `aria-describedby` to link form inputs with error messages.

3. **Keyboard Navigation**: Ensure all interactive elements are reachable via Tab key. Implement focus traps in modals to prevent users from tabbing outside the dialog. Support arrow keys for complex widgets like tabs and dropdown menus.

4. **Focus Management**: Maintain visible focus indicators with `focus:ring` or outline styles. Programmatically return focus to the triggering element when closing modals. Use `useId` from React 18+ to generate stable IDs for `aria-labelledby` and `aria-describedby` associations.

5. **Screen Reader Support**: Include descriptive `alt` text for images (or empty alt for decorative images), use `aria-label` for icon-only buttons, and implement live regions (`aria-live="polite"`) to announce status changes, loading states, and form validation errors.

6. **Form Accessibility**: Associate labels with inputs using `htmlFor`, mark required fields with both visual indicators and `aria-required`, and display errors in close proximity to inputs with `role="alert"` for immediate announcement.

7. **Automated Testing**: Integrate `jest-axe` for unit tests, Storybook's a11y addon for component development, and Lighthouse CI in your deployment pipeline to catch accessibility regressions before they reach production.

## Coming Up Next

**Chapter 33: Analytics & Monitoring**

With your application now accessible to all users, it's essential to understand how they interact with it. In Chapter 33, we'll explore implementing privacy-compliant analytics, setting up real user monitoring (RUM), tracking Core Web Vitals in the field, configuring error tracking with Sentry, and building custom dashboards for application insights. You'll learn how to gather actionable data while respecting user privacy under GDPR and CCPA regulations.